In [31]:
from util import *
import os, shutil
import pandas as pd

from opmap.videoData import VideoData
from opmap.vmemMap import VmemMap
from opmap.phaseMap import PhaseMap
from opmap.phaseVarianceMap import PhaseVarianceMap
from elecpy.elecpySession import ElecpySession

%matplotlib inline

In [32]:
src_dir = '/mnt/recordings/SimulationResult/'
save_dir = '/mnt/recordings/AnalysisResult/Tomii/2018-08-07/'
print(save_dir)
if not os.path.exists(save_dir) : os.mkdir(save_dir)

/mnt/recordings/AnalysisResult/Tomii/2018-08-07/


In [33]:
def analyzeSession(sess, save_dir=None):
    cam = VideoData(*sess.data['vmem'].shape)
    cam.data = - sess.data['vmem']
    v = VmemMap(cam);del(cam)
    p = PhaseMap(v, width = v.data.shape[2]); 
    p.data = - p.data
    pv = PhaseVarianceMap(p, size=3)
    
    if save_dir is not None:
        v.saveImage( os.path.join(save_dir, 'vmem'))
        p.saveImage( os.path.join(save_dir, 'pmap'))
        pv.saveImage( os.path.join(save_dir, 'pvmap'))

    sess.data['phase'] = p.data
    sess.data['pv'] = pv.data

## Check region detail

In [43]:
df = pd.read_csv('data.lr.roi.csv')
df

,sim_id,x_min,x_max,y_min,y_max,f_min,f_max,description
0,20180806-1,90,130,80,120,170,360,pivot
1,20180809-6,73,113,115,155,130,300,shift
2,20180816-6,47,87,99,139,110,280,drift
3,20180817-6,67,107,79,119,10,180,jump
4,20180817-6,134,174,65,105,50,280,jump


In [44]:
thre_phase_integ = 1.5*2*np.pi

In [45]:
for i, row in df.iterrows():
    
    if i not in [0]: continue
    
    dst_dir = os.path.join(save_dir, '{0}'.format(row['sim_id']))
    dst_dir = os.path.join(dst_dir, 'x{0}-{1}_y{2}-{3}_f{4}-{5}'.format(
        row.x_min,row.x_max,
        row.y_min,row.y_max,
        row.f_min,row.f_max,
    ))
    print(dst_dir)
    #if os.path.exists(dst_dir) : shutil.rmtree(dst_dir)
    if not os.path.exists(dst_dir) : os.makedirs(dst_dir)
        
    
    sess = ElecpySession(src_dir+row.sim_id, keys={'vmem'})
    sess.setRange(
        x_min = row.x_min,
        x_max = row.x_max,
        y_min = row.y_min,
        y_max = row.y_max
    )
    
    analyzeSession(sess, save_dir=dst_dir)
    sess.setRange(
        f_min = row.f_min,
        f_max = row.f_max,
    )

    p = sess.data['phase']
    
    ### 3D phase plot ###
    for angle in range(4):

        path = os.path.join(dst_dir,'psurf_{0}'.format(angle))
        if not os.path.exists(path) : os.mkdir(path)

        for frame in range(row.f_max-row.f_min):

            plt.clf()
            plot_phase_surface(sess, frame, angle=angle)
            plt.savefig(os.path.join(path, '{0:0>6}.png'.format(frame)))

    ### integral of time differential ###
    p = sess.data['phase']
    pdiff = copy.deepcopy(p)
    for f in range(len(pdiff)):
        if f == 0: 
            pdiff[f,:,:] = 0
        else:
            pdiff[f,:,:] = p[f,:,:] - p[f-1,:,:]
    pdiff = phaseComplement(pdiff)
    pinteg = abs(np.sum(pdiff, axis=0))
    
    plt.clf()
    plt.imshow(pinteg, vmin=0, cmap='gray') # vmax=2*np.pi, 
    plt.colorbar()
    plt.savefig(os.path.join(dst_dir, 'pinteg.png'))
    
    ### phase graph (time sequence) ###
    pos_y, pos_x = np.where(pinteg > thre_phase_integ)
    neg_y, neg_x = np.where(pinteg <= thre_phase_integ)
    plt.clf()
    plt.subplot(2,1,1)
    for y, x in zip(neg_y, neg_x):
        if x == p.shape[2]//2:
            plt.plot(p[:,y,x], 'b')
    plt.subplot(2,1,2)
    for y, x in zip(pos_y, pos_x):
        if x == p.shape[2]//2:
            plt.plot(p[:,y,x], 'r')
    plt.savefig(os.path.join(dst_dir, 'border_plot.png'))
    plt.close()

    
    ### phase graph (circular plot) ###
    path = os.path.join(dst_dir,'border_plot')
    if not os.path.exists(path) : os.mkdir(path)
    for f in range(len(p)):

        plt.figure(figsize=(5,5))
        plt.clf()

        phases = np.arange(0,2*np.pi,np.pi/1024)
        plt.scatter( np.cos(phases), np.sin(phases), c='k', s=.02)
        
        A = np.arange(-1.2, 1.2, 0.01)
        plt.scatter( A, np.zeros_like(A), c='k', s=.02)
        plt.scatter( np.zeros_like(A), A, c='k', s=.02)

        pos_y, pos_x = np.where(pinteg > thre_phase_integ)
        for y, x in zip(pos_y, pos_x):
            if x == p.shape[2]//2:
                phases = p[f,y,x]
                plt.scatter( np.cos(phases), np.sin(phases), c='r', s=10)

        neg_y, neg_x = np.where(pinteg <= thre_phase_integ)
        for y, x in zip(neg_y, neg_x):
            if x == p.shape[2]//2:
                phases = p[f,y,x]
                plt.scatter( np.cos(phases), np.sin(phases), c='b', s=10)

        plt.axis('equal')
        plt.axis([-1.2, 1.2, -1.2, 1.2])
        plt.savefig(os.path.join(path, '{0:0>6}.png'.format(f)))
        plt.close()

    ### spatial phase differential ###
    path = os.path.join(dst_dir,'pdiff_xy')
    if not os.path.exists(path) : os.mkdir(path)
    for f in range(len(p)):
        
        pdiff_x = np.zeros_like(p[f,:,:], dtype=np.int)
        pdiff_x[0,:] = 0
        pdiff_x[1:,:] = ( np.abs( phaseComplement(p[f,1:,:] - p[f,:-1,:]) ) > (np.pi*2/4)) * 1
        pdiff_y = np.zeros_like(p[f,:,:], dtype=np.int)
        pdiff_y[:,0] = 0
        pdiff_y[:,1:] = ( np.abs( phaseComplement(p[f,:,1:] - p[f,:,:-1]) ) > (np.pi*2/4)) * 1
        
        pdiff_xy = pdiff_x+pdiff_y
        
        plt.clf()
        plt.imshow(pdiff_xy, vmin=0, vmax=2, cmap='gray')
        plt.savefig(os.path.join(path, '{0:0>6}.png'.format(f)))
        plt.close()
        
        
#     ### Sodium current integral ###
    
#     xina = abs(np.sum(sess.data["cell/xina"], axis=0))
#     plt.clf()
#     plt.imshow(xina, cmap='gray')
#     plt.colorbar()
#     plt.savefig(os.path.join(dst_dir, 'xina.png'))
    
#     plt.clf()
#     plt.imshow(np.mean(sess.data['pv'] > 0.3, axis=0), cmap='gray')
#     plt.colorbar()
#     plt.savefig(os.path.join(dst_dir, 'traj.png'))
    


/mnt/recordings/AnalysisResult/Tomii/2018-08-07/20180806-1/x90-130_y80-120_f170-360
